In [8]:
# 1. Import libraries and config
import pandas as pd
import numpy as np
import os
import sys

# --- Add Project Root to Python Path ---
# This allows us to import from 'src'
PROJECT_ROOT = '/content/drive/MyDrive/NFL_Prediction_System' # Adjust if_needed
if PROJECT_ROOT not in sys.path:
    sys.path.append(str(PROJECT_ROOT))
# ---------------------------------------

from src.utils import config  # Import our config file

# Make sure the output directory exists
os.makedirs(config.CLEANED_GAMES_PATH, exist_ok=True)

print("Libraries and config paths set.")
print(f"Loading data from: {config.RAW_GAMES_PATH}")
print(f"Will save cleaned data to: {config.CLEANED_GAMES_PATH}")

Libraries and config paths set.
Loading data from: /content/drive/MyDrive/NFL_Prediction_System/data/raw/games
Will save cleaned data to: /content/drive/MyDrive/NFL_Prediction_System/data/cleaned/games


In [9]:
# 2. Load Raw Data
print("Loading raw data (2002-2023)...")
try:
    # --- UPDATE: Point to the new 2002-2023 files ---
    pbp_df = pd.read_parquet(config.RAW_GAMES_PATH / 'pbp_2002_2023.parquet')
    schedule_df = pd.read_csv(config.RAW_GAMES_PATH / 'schedule_2002_2023.csv')

    print(f"Loaded PBP data: {pbp_df.shape}")
    print(f"Loaded schedule data: {schedule_df.shape}")

except FileNotFoundError as e:
    print(f"Error: {e}")
    print("Please ensure '01_data_ingestion.ipynb' ran successfully with the 2002-2023 range.")

Loading raw data (2002-2023)...
Loaded PBP data: (1044769, 20)
Loaded schedule data: (5929, 46)


In [10]:
# 3. Clean PBP Data (Filter for relevant plays)
# This logic is correct and does not need to change.
print(f"Original PBP shape: {pbp_df.shape}")

pbp_filtered = pbp_df[
    (pbp_df['season_type'] == 'REG') &
    (pbp_df['epa'].notna()) &
    (pbp_df['play_type'].isin(['pass', 'run']))
].copy()

print(f"Filtered PBP shape (meaningful plays only): {pbp_filtered.shape}")

Original PBP shape: (1044769, 20)
Filtered PBP shape (meaningful plays only): (711337, 20)


In [11]:
# 4. Aggregate PBP Stats to Team-Game Level
# This logic is correct and does not need to change.
print("Aggregating PBP data to team-game level...")

# Create new columns for pass_epa and run_epa *before* grouping
# Ensure we use 'play_type' because 'pass'/'run' columns might not exist in the new file
pbp_filtered['pass_epa'] = pbp_filtered['epa'].where(pbp_filtered['play_type'] == 'pass')
pbp_filtered['run_epa'] = pbp_filtered['epa'].where(pbp_filtered['play_type'] == 'run')

# Aggregate offensive stats
offensive_stats = pbp_filtered.groupby(['game_id', 'posteam']).agg(
    off_epa_per_play=('epa', 'mean'),
    off_success_rate=('success', 'mean'),
    off_pass_epa=('pass_epa', 'mean'),
    off_run_epa=('run_epa', 'mean'),
    off_interceptions=('interception', 'sum'),
    off_fumbles_lost=('fumble_lost', 'sum')
).reset_index()

# Create the total turnovers column
offensive_stats['off_turnovers'] = offensive_stats['off_interceptions'] + offensive_stats['off_fumbles_lost']

print("Offensive stats calculated.")

# Aggregate defensive stats
defensive_stats = offensive_stats.rename(columns={
    'posteam': 'defteam',
    'off_epa_per_play': 'def_epa_per_play',
    'off_success_rate': 'def_success_rate',
    'off_pass_epa': 'def_pass_epa',
    'off_run_epa': 'def_run_epa',
    'off_turnovers': 'def_turnovers_forced'
})

defensive_stats = defensive_stats.drop(columns=['off_interceptions', 'off_fumbles_lost'])
print("Defensive stats calculated.")

Aggregating PBP data to team-game level...
Offensive stats calculated.
Defensive stats calculated.


In [12]:
# 5. Clean Schedule Data (Create Target Variables)
# This logic is correct and does not need to change.
print("Cleaning schedule data and creating target variables...")

schedule_clean = schedule_df[schedule_df['game_type'] == 'REG'][[
    'game_id', 'season', 'week', 'home_team', 'away_team',
    'home_score', 'away_score', 'spread_line', 'total_line'
]].copy()

# Create our primary target variables
schedule_clean['total_score'] = schedule_clean['home_score'] + schedule_clean['away_score']
schedule_clean['home_win'] = (schedule_clean['home_score'] > schedule_clean['away_score']).astype(int)

# Create betting-related target variables
schedule_clean['home_cover'] = (schedule_clean['home_score'] - schedule_clean['away_score'] + schedule_clean['spread_line'] > 0).astype(int)
schedule_clean['over_hit'] = (schedule_clean['total_score'] > schedule_clean['total_line']).astype(int)

print("Target variables created.")

Cleaning schedule data and creating target variables...
Target variables created.


In [13]:
# 6. Merge Stats into Final Game-Level Dataset
# This logic is correct and does not need to change.
print("Merging stats into final game-level dataset...")

# Merge home team offensive stats
game_data = pd.merge(
    schedule_clean,
    offensive_stats.rename(columns={'posteam': 'home_team'}),
    on=['game_id', 'home_team'],
    how='left'
)
# Merge home team defensive stats
game_data = pd.merge(
    game_data,
    defensive_stats.rename(columns={'defteam': 'home_team'}),
    on=['game_id', 'home_team'],
    how='left'
)

# Merge away team offensive stats
game_data = pd.merge(
    game_data,
    offensive_stats.rename(columns={'posteam': 'away_team',
                                    'off_epa_per_play': 'away_off_epa_per_play',
                                    'off_success_rate': 'away_off_success_rate',
                                    'off_pass_epa': 'away_off_pass_epa',
                                    'off_run_epa': 'away_off_run_epa',
                                    'off_turnovers': 'away_off_turnovers'}),
    on=['game_id', 'away_team'],
    how='left'
)
# Merge away team defensive stats
game_data = pd.merge(
    game_data,
    defensive_stats.rename(columns={'defteam': 'away_team',
                                    'def_epa_per_play': 'away_def_epa_per_play',
                                    'def_success_rate': 'away_def_success_rate',
                                    'def_pass_epa': 'away_def_pass_epa',
                                    'def_run_epa': 'away_def_run_epa',
                                    'def_turnovers_forced': 'away_def_turnovers_forced'}),
    on=['game_id', 'away_team'],
    how='left'
)

# Rename home columns for clarity
game_data = game_data.rename(columns={
    'off_epa_per_play': 'home_off_epa_per_play',
    'off_success_rate': 'home_off_success_rate',
    'off_pass_epa': 'home_off_pass_epa',
    'off_run_epa': 'home_off_run_epa',
    'off_turnovers': 'home_off_turnovers',
    'def_epa_per_play': 'home_def_epa_per_play',
    'def_success_rate': 'home_def_success_rate',
    'def_pass_epa': 'home_def_pass_epa',
    'def_run_epa': 'home_def_run_epa',
    'def_turnovers_forced': 'home_def_turnovers_forced'
})

print("Final merged dataset (one row per game):")
print(game_data.head())
print(f"Final shape: {game_data.shape}")

Merging stats into final game-level dataset...
Final merged dataset (one row per game):
           game_id  season  week home_team away_team  home_score  away_score  \
0   2002_01_SF_NYG    2002     1       NYG        SF          13          16   
1  2002_01_NYJ_BUF    2002     1       BUF       NYJ          31          37   
2  2002_01_BAL_CAR    2002     1       CAR       BAL          10           7   
3  2002_01_MIN_CHI    2002     1       CHI       MIN          27          23   
4   2002_01_SD_CIN    2002     1       CIN        SD           6          34   

   spread_line  total_line  total_score  ...  away_off_pass_epa  \
0         -4.0        39.0           29  ...           0.077821   
1         -3.0        43.0           68  ...           0.180922   
2          0.0        33.0           17  ...          -0.027602   
3          4.5        41.0           50  ...           0.295529   
4          3.0        37.0           40  ...                NaN   

   away_off_run_epa  off_int

In [14]:
# 7. Save Cleaned Data
# We'll name this 'game_data_2002_2023.parquet' to reflect the huge new range
save_path = config.CLEANED_GAMES_PATH / 'game_data_2002_2023.parquet'

game_data.to_parquet(save_path, index=False)

print(f"\nSuccessfully cleaned and aggregated data.")
print(f"Saved to: {save_path}")


Successfully cleaned and aggregated data.
Saved to: /content/drive/MyDrive/NFL_Prediction_System/data/cleaned/games/game_data_2002_2023.parquet
